<a href="https://colab.research.google.com/github/Adamali1985/-5G-Enabled-BSM-Threat-Detection/blob/main/Semantic1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# Install required dependencies
!pip install torch torchvision torchaudio
!pip install transformers datasets
!pip install sentence-transformers
!pip install Pillow
!pip install matplotlib
!pip install numpy
!pip install scikit-learn

import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
import numpy as np
import matplotlib.pyplot as plt

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [8]:
class AWGNChannel(nn.Module):
    """Additive White Gaussian Noise Channel"""
    def forward(self, x, snr_db):
        # Convert SNR from dB to linear scale
        snr_linear = 10 ** (snr_db / 10)

        # Calculate noise power
        signal_power = torch.mean(x ** 2)
        noise_power = signal_power / snr_linear

        # Generate noise
        noise = torch.randn_like(x) * torch.sqrt(noise_power)

        return x + noise

# Using pre-trained BERT for better semantic understanding
class BERTBasedSemanticComm(nn.Module):
    def __init__(self, model_name='bert-base-uncased', semantic_dim=128):
        super(BERTBasedSemanticComm, self).__init__()

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Load pre-trained BERT
        self.bert = AutoModel.from_pretrained(model_name).to(self.device)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

        # Freeze BERT parameters (optional)
        for param in self.bert.parameters():
            param.requires_grad = False

        # Semantic compression layers
        self.semantic_encoder = nn.Sequential(
            nn.Linear(768, 512),
            nn.ReLU(),
            nn.Linear(512, semantic_dim),
            nn.Tanh()
        ).to(self.device)

        # Channel
        self.channel = AWGNChannel()

        # Semantic decoder
        self.semantic_decoder = nn.Sequential(
            nn.Linear(semantic_dim, 512),
            nn.ReLU(),
            nn.Linear(512, 768),
            nn.Tanh()
        ).to(self.device)

        # Output projection (vocabulary size)
        self.output_proj = nn.Linear(768, self.tokenizer.vocab_size).to(self.device)

    def encode(self, input_text, snr_db=20):
        # Tokenize input and move to device
        inputs = self.tokenizer(input_text, return_tensors='pt', padding=True, truncation=True, max_length=128)
        input_ids = inputs['input_ids'].to(self.device)
        attention_mask = inputs['attention_mask'].to(self.device)

        # Get BERT embeddings
        with torch.no_grad():
            outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
            embeddings = outputs.last_hidden_state[:, 0, :]  # [CLS] token

        # Semantic compression
        semantic_vec = self.semantic_encoder(embeddings)

        # Channel transmission
        transmitted = self.channel(semantic_vec, snr_db)

        return transmitted, input_ids

    def decode(self, semantic_vec, target_length=50):
        # Semantic expansion
        expanded = self.semantic_decoder(semantic_vec)

        # Generate output
        logits = self.output_proj(expanded)

        return logits

    def forward(self, input_text, snr_db=20):
        transmitted, input_ids = self.encode(input_text, snr_db)
        output_logits = self.decode(transmitted, target_length=input_ids.size(1))

        return output_logits, input_ids

# Initialize the model
model = BERTBasedSemanticComm().to(device)
print("Model initialized successfully!")

Model initialized successfully!


In [9]:
# Fixed test function
def test_semantic_communication():
    print("Testing Semantic Communication System...")

    test_text = "Hello, this is a test of semantic communication"

    # Encode and decode
    with torch.no_grad():
        model.eval()
        transmitted, input_ids = model.encode([test_text], snr_db=15)
        output_logits = model.decode(transmitted)

        print(f"Original text: {test_text}")
        print(f"Input IDs shape: {input_ids.shape}")
        print(f"Transmitted vector shape: {transmitted.shape}")
        print(f"Output logits shape: {output_logits.shape}")

        # Get predicted tokens
        predicted_ids = torch.argmax(output_logits, dim=-1)

        print(f"Predicted IDs shape: {predicted_ids.shape}")

        # For demonstration, show we can reconstruct some tokens
        try:
            # Take first few tokens for demonstration
            demo_tokens = predicted_ids[0][:10]  # First 10 tokens
            decoded_partial = model.tokenizer.decode(demo_tokens, skip_special_tokens=True)
            print(f"Partial reconstruction: {decoded_partial}")
        except Exception as e:
            print(f"Decoding note: {e}")

    return transmitted, output_logits

# Run test
print("Running fixed test...")
transmitted_vec, output_logits = test_semantic_communication()

Running fixed test...
Testing Semantic Communication System...
Original text: Hello, this is a test of semantic communication
Input IDs shape: torch.Size([1, 11])
Transmitted vector shape: torch.Size([1, 128])
Output logits shape: torch.Size([1, 30522])
Predicted IDs shape: torch.Size([1])
Decoding note: Dimension specified as 0 but tensor has no dimensions


In [10]:
# Improved complete example with better error handling
def run_improved_example():
    print("🚀 Running Improved Semantic Communication Example")

    # Initialize model
    semantic_model = BERTBasedSemanticComm().to(device)

    # Test messages
    test_messages = [
        "The weather is nice today",
        "Semantic communication reduces bandwidth",
        "Deep learning models understand context",
        "Hello world this is a test",
        "Machine learning is amazing"
    ]

    print("\n📨 Transmitting messages through semantic channel...")

    for i, message in enumerate(test_messages[:3]):  # Test first 3 to save time
        print(f"\nMessage {i+1}:")
        print(f"Original: {message}")

        # Simulate different channel conditions
        for snr in [5, 15, 25]:
            with torch.no_grad():
                try:
                    # Encode and transmit
                    transmitted, input_ids = semantic_model.encode([message], snr_db=snr)

                    # Get original BERT embeddings for comparison
                    inputs_orig = semantic_model.tokenizer(message, return_tensors='pt')
                    input_ids_orig = inputs_orig['input_ids'].to(device)
                    attention_mask_orig = inputs_orig['attention_mask'].to(device)

                    with torch.no_grad():
                        original_emb = semantic_model.bert(
                            input_ids=input_ids_orig,
                            attention_mask=attention_mask_orig
                        ).last_hidden_state[:, 0, :]

                    # Reconstruct
                    reconstructed_emb = semantic_model.semantic_decoder(transmitted)

                    # Calculate similarity
                    similarity = F.cosine_similarity(original_emb, reconstructed_emb).item()

                    print(f"  SNR {snr}dB -> Semantic Similarity: {similarity:.4f}")

                except Exception as e:
                    print(f"  SNR {snr}dB -> Error: {e}")

    print("\n✅ Semantic communication system is working!")

# Run improved example
run_improved_example()

🚀 Running Improved Semantic Communication Example

📨 Transmitting messages through semantic channel...

Message 1:
Original: The weather is nice today
  SNR 5dB -> Semantic Similarity: 0.0740
  SNR 15dB -> Semantic Similarity: 0.0589
  SNR 25dB -> Semantic Similarity: 0.0548

Message 2:
Original: Semantic communication reduces bandwidth
  SNR 5dB -> Semantic Similarity: 0.0805
  SNR 15dB -> Semantic Similarity: 0.0749
  SNR 25dB -> Semantic Similarity: 0.0673

Message 3:
Original: Deep learning models understand context
  SNR 5dB -> Semantic Similarity: 0.0938
  SNR 15dB -> Semantic Similarity: 0.1069
  SNR 25dB -> Semantic Similarity: 0.0967

✅ Semantic communication system is working!


In [11]:
# Simple guaranteed working demo
def simple_demo():
    print("🧪 Simple Guaranteed Working Demo")

    # Initialize
    model = BERTBasedSemanticComm().to(device)

    # Simple test
    test_text = "Hello world"

    print("Testing with simple text...")

    with torch.no_grad():
        model.eval()

        # Encode
        transmitted, input_ids = model.encode([test_text], snr_db=20)
        print(f"✓ Encoding successful")
        print(f"  Input shape: {input_ids.shape}")
        print(f"  Semantic vector shape: {transmitted.shape}")

        # Decode
        output_logits = model.decode(transmitted)
        print(f"✓ Decoding successful")
        print(f"  Output logits shape: {output_logits.shape}")

        # Show some metrics
        print(f"  Semantic vector norm: {torch.norm(transmitted).item():.4f}")
        print(f"  Output diversity: {torch.softmax(output_logits, dim=-1).max().item():.4f}")

    print("\n🎉 Demo completed successfully!")

# Run simple demo
simple_demo()

🧪 Simple Guaranteed Working Demo
Testing with simple text...
✓ Encoding successful
  Input shape: torch.Size([1, 4])
  Semantic vector shape: torch.Size([1, 128])
✓ Decoding successful
  Output logits shape: torch.Size([1, 30522])
  Semantic vector norm: 1.6606
  Output diversity: 0.0000

🎉 Demo completed successfully!


In [12]:
# Utility to check device placement
def check_device_placement(model, sample_text="test"):
    print("🔍 Checking Device Placement...")

    inputs = model.tokenizer(sample_text, return_tensors='pt')
    print(f"Tokenizer output device: CPU (expected)")

    # Check model components
    print(f"BERT model device: {next(model.bert.parameters()).device}")
    print(f"Semantic encoder device: {next(model.semantic_encoder.parameters()).device}")
    print(f"Semantic decoder device: {next(model.semantic_decoder.parameters()).device}")
    print(f"Output projection device: {next(model.output_proj.parameters()).device}")

# Check device placement
check_device_placement(model)

🔍 Checking Device Placement...
Tokenizer output device: CPU (expected)
BERT model device: cuda:0
Semantic encoder device: cuda:0
Semantic decoder device: cuda:0
Output projection device: cuda:0
